In [1]:
import pandas as pd
import re
import transformers as ppb
import numpy as np
from scipy import sparse

In [2]:
DATA_DIR = "./data"
admissions_file = DATA_DIR + '/ADMISSIONS.csv'

In [12]:
full_admissions = pd.read_csv(admissions_file)
print(full_admissions.shape)
print(full_admissions.dtypes)
full_admissions.head()

(58976, 19)
ROW_ID                   int64
SUBJECT_ID               int64
HADM_ID                  int64
ADMITTIME               object
DISCHTIME               object
DEATHTIME               object
ADMISSION_TYPE          object
ADMISSION_LOCATION      object
DISCHARGE_LOCATION      object
INSURANCE               object
LANGUAGE                object
RELIGION                object
MARITAL_STATUS          object
ETHNICITY               object
EDREGTIME               object
EDOUTTIME               object
DIAGNOSIS               object
HOSPITAL_EXPIRE_FLAG     int64
HAS_CHARTEVENTS_DATA     int64
dtype: object


,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [15]:
full_admissions['ADMITTIME'] = pd.to_datetime(full_admissions['ADMITTIME'])
full_admissions['DISCHTIME'] = pd.to_datetime(full_admissions['DISCHTIME'])
full_admissions['DEATHTIME'] = pd.to_datetime(full_admissions['DEATHTIME'])
full_admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [19]:
admits_patient_df = full_admissions.sort_values(by=['SUBJECT_ID','DISCHTIME']).loc[:,('ROW_ID','SUBJECT_ID','HADM_ID','ADMITTIME',\
    'DISCHTIME','DEATHTIME','ADMISSION_TYPE','DIAGNOSIS','HOSPITAL_EXPIRE_FLAG','HAS_CHARTEVENTS_DATA')]
admits_patient_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
211,1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NaT,NEWBORN,NEWBORN,0,1
212,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,HYPOTENSION,0,1
213,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1
214,4,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NaT,NEWBORN,NEWBORN,0,1
215,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,CHRONIC RENAL FAILURE/SDA,0,1


In [90]:
mult_admits = admits_patient_df[admits_patient_df.duplicated(subset=['SUBJECT_ID'],keep=False)]
mult_admits.head(15)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
224,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaT,ELECTIVE,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1
225,15,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,NaT,EMERGENCY,PERICARDIAL EFFUSION,0,1
229,19,21,109451,2134-09-11 12:17:00,2134-09-24 16:15:00,NaT,EMERGENCY,CONGESTIVE HEART FAILURE,0,1
230,20,21,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,2135-02-08 02:08:00,EMERGENCY,SEPSIS,1,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,BRAIN MASS,0,1
12,33,34,115799,2186-07-18 16:46:00,2186-07-20 16:00:00,NaT,EMERGENCY,CHEST PAIN\CATH,0,1
13,34,34,144319,2191-02-23 05:23:00,2191-02-25 20:20:00,NaT,EMERGENCY,BRADYCARDIA,0,1
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaT,EMERGENCY,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaT,EMERGENCY,CHEST PAIN/SHORTNESS OF BREATH,0,1


In [84]:
mult_admits[mult_admits['SUBJECT_ID']==36].loc[:,('DISCHTIME','READMIT')].rolling(window='30D', on='DISCHTIME',closed='left').sum()

,DISCHTIME,READMIT
15,2131-05-08 14:00:00,NaN
16,2131-05-25 13:30:00,1.0
17,2134-05-20 13:16:00,NaN


In [91]:
subjects = mult_admits['SUBJECT_ID'].unique()
mult_admits.loc[:,'READMIT'] = 1

# identify re-admits within 30 days
for subject in subjects:
    readmitted = mult_admits[mult_admits['SUBJECT_ID'] == subject].loc[:,('DISCHTIME','READMIT')].rolling(window='30D', on='DISCHTIME',closed='left').sum()
    #print(readmitted)
    mult_admits.loc[readmitted.index, 'READMIT'] = readmitted['READMIT']
    #print(subject)

mult_admits['READMIT_2'] = mult_admits['READMIT'].shift(-1)

/home/npodpx/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/npodpx/venv/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/tmp/ipykernel_23941/3362247210.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [92]:
mult_admits[mult_admits['SUBJECT_ID'] == 36]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,READMIT,READMIT_2
15,36,36,182104,2131-04-30 07:15:00,2131-05-08 14:00:00,NaT,EMERGENCY,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,NaN,1.0
16,37,36,122659,2131-05-12 19:49:00,2131-05-25 13:30:00,NaT,EMERGENCY,CHEST PAIN/SHORTNESS OF BREATH,0,1,1.0,NaN
17,38,36,165660,2134-05-10 11:30:00,2134-05-20 13:16:00,NaT,ELECTIVE,VENTRAL HERNIA/SDA,0,1,NaN,NaN
